In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session
from IPython.display import clear_output

In [ ]:
import requests

def telegram_message(message, bot_token, chat_id):
    """
    Send a message using the Telegram bot.
    
    Parameters:
    - message (str): The message to send.
    - bot_token (str): The token for the Telegram bot.
    - chat_id (int): The chat ID to send the message to.
    """
    url = f"https://api.telegram.org/bot{bot_token}/sendMessage"
    data = {"chat_id": chat_id, "text": message}
    response = requests.post(url, data=data)
    if response.status_code == 200:
        print(f"Message sent: {message}")
    else:
        print("Failed to send message:", response.text)

# Example usage
bot_token = "your_bot_token_here"  # Replace with your bot token
chat_id = 123456789  # Replace with your chat ID
telegram_message("📋 Bloggy Automation started 🚀🚀🚀🚀", bot_token, chat_id)


In [ ]:
# Install necessary libraries
! pip install feedparser requests beautifulsoup4 sentence-transformers scikit-learn google-auth google-auth-oauthlib google-auth-httplib2 google-api-python-client boto3 pillow
!pip install diffusers transformers accelerate scipy safetensors
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117


clear_output()

In [ ]:
# Step 1: Import necessary libraries
import feedparser
import requests
from bs4 import BeautifulSoup
import json
import time
import random
import os
from datetime import datetime
import threading
import subprocess
import logging
import re
from huggingface_hub import login
from diffusers import StableDiffusionPipeline # Add EulerDiscreteScheduler here
import torch
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
logging.getLogger("requests").setLevel(logging.WARNING)


In [ ]:


# Step 2: Fetch and Filter Articles from Different Sources

# Function to fetch and filter RSS feed data from Towards Data Science
def fetch_rss_feed(url):
    articles = []
    try:
        feed = feedparser.parse(url)
        restricted_phrases = ["Continue reading on Towards Data Science", "source=rss"]
        
        for entry in feed.entries[:20]:  # Limit to latest 20 entries
            if not any(phrase in entry.summary for phrase in restricted_phrases):
                summary_soup = BeautifulSoup(entry.summary, "html.parser")
                clean_summary = summary_soup.get_text(separator=" ", strip=True)
                
                articles.append({
                    'title': entry.title,
                    'link': entry.link,
                    'content': clean_summary
                })
    except Exception as e:
        print(f"Error fetching RSS feed: {e}")
    
    return articles

# Function to fetch articles from KDNuggets
def fetch_kdnuggets_articles():
    base_url = "https://www.kdnuggets.com/"
    headers = {'User-Agent': 'Mozilla/5.0'}
    articles = []

    try:
        response = requests.get(base_url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            for a_tag in soup.find_all('a'):
                b_tag = a_tag.find('b')
                if b_tag:
                    title = b_tag.get_text(strip=True)
                    link = a_tag.get('href')
                    if link and not link.startswith('http'):
                        link = base_url + link
                    
                    try:
                        page_response = requests.get(link, headers=headers)
                        page_soup = BeautifulSoup(page_response.content, 'html.parser')
                        post_div = page_soup.find('div', id='post-')
                        content = post_div.get_text(strip=True) if post_div else "Content not found"
                        
                        articles.append({
                            'title': title,
                            'link': link,
                            'content': content
                        })
                    except Exception as e:
                        print(f"Error fetching KDNuggets article at {link}: {e}")
                    time.sleep(1 + random.uniform(0, 1))
    except Exception as e:
        print(f"Error fetching KDNuggets homepage: {e}")
    
    return articles

# Function to fetch articles from Dev.to
def fetch_devto_articles():
    url = "https://dev.to/t/ai/latest"
    headers = {'User-Agent': 'Mozilla/5.0'}
    articles = []

    try:
        response = requests.get(url, headers=headers)
        soup = BeautifulSoup(response.content, "html.parser")
        
        for article in soup.find_all("div", class_="crayons-story"):
            title_tag = article.find("h2", class_="crayons-story__title")
            link_tag = title_tag.find("a") if title_tag else None
            
            if title_tag and link_tag:
                title = title_tag.get_text(strip=True)
                link = f"https://dev.to{link_tag['href']}"
                
                try:
                    article_response = requests.get(link, headers=headers)
                    article_soup = BeautifulSoup(article_response.content, "html.parser")
                    content_div = article_soup.find("div", class_="crayons-article__body text-styles spec__body")
                    content = content_div.get_text(strip=True) if content_div else "Content not found"
                    
                    articles.append({
                        'title': title,
                        'link': link,
                        'content': content
                    })
                except Exception as e:
                    print(f"Error fetching Dev.to article at {link}: {e}")
                
                time.sleep(1 + random.uniform(0, 1))
    except Exception as e:
        print(f"Error fetching Dev.to homepage: {e}")

    return articles

# Function to fetch articles from NVIDIA blog
def fetch_nvidia_blog_articles():
    url = "https://developer.nvidia.com/blog/recent-posts/"
    headers = {'User-Agent': 'Mozilla/5.0'}
    articles = []

    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            soup = BeautifulSoup(response.content, 'html.parser')
            for link_tag in soup.find_all('a', class_='carousel-row-slide__link'):
                title_span = link_tag.find('span', class_='visually-hidden')
                if title_span:
                    title = title_span.get_text(strip=True)
                    link = link_tag['href']
                    if not link.startswith('http'):
                        link = 'https://developer.nvidia.com' + link
                    
                    try:
                        post_response = requests.get(link, headers=headers)
                        post_soup = BeautifulSoup(post_response.content, 'html.parser')
                        content_div = post_soup.find('div', class_='entry-content')
                        content = content_div.get_text(separator="\n", strip=True) if content_div else "Content not found"
                        
                        articles.append({
                            'title': title,
                            'link': link,
                            'content': content
                        })
                    except Exception as e:
                        print(f"Error fetching NVIDIA article at {link}: {e}")
                    
                    time.sleep(1 + random.uniform(0, 1))
    except Exception as e:
        print(f"Error fetching NVIDIA homepage: {e}")

    return articles

# Function to save articles to JSON with timestamp
def save_articles_to_json():
    all_articles = {
        "Towards Data Science": fetch_rss_feed("https://towardsdatascience.com/feed"),
        "KDNuggets": fetch_kdnuggets_articles(),
        "Dev.to": fetch_devto_articles(),
        "NVIDIA Blog": fetch_nvidia_blog_articles()
    }
    
    current_time = datetime.now().strftime("%d-%m-%y-%H-%M")
    file_name = f"{current_time}.json"
    
    try:
        with open(file_name, "w") as json_file:
            json.dump(all_articles, json_file, indent=4)
        print(f"Articles saved to '{file_name}'.")
        return file_name
    except Exception as e:
        print(f"Error saving articles to JSON: {e}")
        return None

# Run to save and get the file path
saved_file_path = save_articles_to_json()


telegram_message("Documentation gathered successfully! 📑✅")

In [ ]:

# Function to load JSON data
def load_json(file_path):
    try:
        with open(file_path, 'r') as f:
            data = json.load(f)
        print("Data loaded successfully.")
        return data
    except Exception as e:
        print(f"Error loading JSON file: {e}")
        return None

# Load data using the saved file path
if saved_file_path:
    data = load_json(f"/kaggle/working/{saved_file_path}")
else:
    print("No file path available to load data.")


In [ ]:
# Install necessary dependencies
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh
clear_output()

In [ ]:
# Create a Python function to start the Ollama API server in a separate thread with logging suppression
def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    # Redirect stderr to null to suppress GIN logs
    subprocess.Popen(["ollama", "serve"], stderr=subprocess.DEVNULL)

# Start the Ollama server in a separate thread
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

# Pull the model (this will suppress output as well)
!ollama pull llama3.2:3b 
clear_output()

In [ ]:
# Define the llama function
def llama(prompt_or_messages, temperature, max_tokens, raw=False, debug=False):
    # Define the model name based on the provided model_size
    model = 'llama3.2:3b'
    
    # Determine if input is a string (prompt) or list (messages for chat)
    if isinstance(prompt_or_messages, str):
        prompt = prompt_or_messages
        url = "http://127.0.0.1:11434/v1/completions"
        payload = {
            "model": model,
            "temperature": temperature,
            "prompt": prompt,
            "max_tokens": max_tokens
        }
    else:
        messages = prompt_or_messages
        url = "http://127.0.0.1:11434/v1/chat/completions"
        payload = {
            "model": model,
            "temperature": temperature,
            "max_tokens": max_tokens,
            "stop": ["<|eot_id|>", "<|eom_id|>"],
            "messages": messages
        }

    if debug:
        print("Payload:", payload)
    
    headers = {
        "Accept": "application/json",
        "Content-Type": "application/json"
    }

    try:
        response = requests.post(url, headers=headers, data=json.dumps(payload))
        response.raise_for_status()  # Raises HTTPError for bad responses
        res = response.json()
    except requests.exceptions.RequestException as e:
        raise Exception(f"Request failed: {e}")

    if 'error' in res:
        raise Exception(f"API Error: {res['error']}")

    if raw:
        return res

    # Return the relevant response depending on prompt or message input
    if isinstance(prompt_or_messages, str):
        return res['choices'][0].get('text', '')
    else:
        return res['choices'][0].get('message', {}).get('content', '')


In [ ]:
# Log in to Hugging Face Hub
huggingface_token = "your_huggingface_token_here"  # Replace with your actual Hugging Face token
login(token=huggingface_token)

# Model and device setup
model_id = "CompVis/stable-diffusion-v1-4"
device = "cuda"

pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)
pipe = pipe.to(device)

clear_output()

print("done")

telegram_message("LLM model downloaded successfully! 🤖✅")

In [ ]:
system_message = "Extract the main topic or keyword from the following article content and summarize it in one sentence."

def extract_topics(data):
    for publication, articles in data.items():  # Access each publication key and its article list
        for article in articles:  # Iterate over each article within the publication
            content = article.get("content", "")
            if content:
                # Use the first 500 characters of content if available
                truncated_content = content[:500]
                
                # OpenAI API request to extract main topic from content
                try:
                    prompt = (
                                "<|begin_of_text|>"
                                "<|start_header_id|>system<|end_header_id|>"
                                f"{system_message}"
                                "<|eot_id|>"    
                                "<|start_header_id|>user<|end_header_id|>"
                                f"{truncated_content}...."
                                "<|eot_id|>"
                                "<|start_header_id|>assistant<|end_header_id|>"
                            )
                    
                    # Get the response text and store it in a new field in data
                    main_topic = llama(prompt, max_tokens=4096, temperature=0.3)
                    
                    article["main_topic"] = main_topic
                except Exception as e:
                    print(f"Error processing content '{truncated_content}': {e}")
                    article["main_topic"] = "Error in extraction"
    return data

# Extract topics from content and add to the data structure
data_with_topics = extract_topics(data)
clear_output()


In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

# Load the all-MiniLM-L6-v2 model
model = SentenceTransformer('all-MiniLM-L6-v2')

def generate_embeddings(data):
    embeddings = []
    topics = []
    for publication, articles in data.items():
        for article in articles:
            main_topic = article.get("main_topic")
            if main_topic and "Error" not in main_topic:  # Ignore errors
                # Generate the embedding for the main topic
                embedding = model.encode(main_topic)
                embeddings.append(embedding)
                topics.append(main_topic)
    
    return embeddings, topics

# Generate embeddings and store topics
embeddings, topics = generate_embeddings(data_with_topics)

clear_output()

In [ ]:
from sklearn.cluster import KMeans

# Convert embeddings to a 2D array if it is 1D
if len(np.array(embeddings).shape) == 1:
    embeddings = np.array(embeddings).reshape(-1, 1)

# Set the number of clusters
num_clusters = 3
kmeans = KMeans(n_clusters=num_clusters, random_state=0)
labels = kmeans.fit_predict(embeddings)

# Associate each topic with a cluster label
clustered_topics = {i: [] for i in range(num_clusters)}
for idx, label in enumerate(labels):
    clustered_topics[label].append(topics[idx])

# Print clustered topics for each group
for cluster, topic_list in clustered_topics.items():
    print(f"Cluster {cluster}: {topic_list}")

In [ ]:
#Calculate Scores for Each Topic
import random  # Simulate scores (replace this with actual logic)

def calculate_relevance_score(topic):
    # Placeholder function to simulate scoring based on frequency, recency, etc.
    # In practice, you would calculate based on actual data metrics.
    return random.uniform(0, 1)  # Replace with actual scoring logic

# Calculate scores for each topic and select the top topics across clusters
topic_scores = []

for cluster, topic_list in clustered_topics.items():
    for topic in topic_list:
        relevance_score = calculate_relevance_score(topic)
        topic_scores.append((topic, relevance_score))

# Sort by relevance score in descending order and select the top 5
top_topics = sorted(topic_scores, key=lambda x: x[1], reverse=True)[:5]

# Display the top 5 topics
print("Top 5 Topics:")
for idx, (topic, score) in enumerate(top_topics, 1):
    print(f"Top {idx} Topic: '{topic}' with relevance score {score:.2f}")
    print('\n')

In [ ]:
# Function to retrieve content for a given topic
def get_content_for_topic(data, topic):
    for publication, articles in data.items():
        for article in articles:
            if article.get("main_topic") == topic:
                return article.get("content", "")
    return ""
# Generate blog sections with reference to content and structure
def generate_blog_with_references(top_topic, data):
    # Retrieve content associated with the top topic
    topic_content = get_content_for_topic(data, top_topic)
    
    if not topic_content:
        print(f"No content found for topic: {top_topic}")
        return
    
    # Define the prompt for generating the blog post
    system_message = (
        "You will be given a topic title and content to reference. Write a detailed and engaging blog post based on this material at your own. "
        "Begin with an introduction that hooks the reader and provides an overview of the topic. Break the main content into clear, well-organized sections,also add what will bo good for it, "
        "each exploring a key aspect or subtopic from the reference content. Add any relevant examples, actionable tips, or recent data to enrich the post "
        "and make it practical for readers.\n\nWrite in a friendly, conversational tone that draws readers in, aiming for a length of about 1000-1,200 words. "
        "Conclude with a summary of the key points and a call-to-action that encourages readers to engage further, such as by sharing their thoughts or exploring related resources."
    )

    # Format the messages for the initial blog generation
   
    question = f"Write blog with this Reference Topic: {top_topic}.\n Content: {topic_content[:650]}..."
    prompt = (
        "<|begin_of_text|>"
        "<|start_header_id|>system<|end_header_id|>"
        f"{system_message}"
        "<|eot_id|>"    
        "<|start_header_id|>user<|end_header_id|>"
        f"{question}"
        "<|eot_id|>"
        "<|start_header_id|>assistant<|end_header_id|>"
    )
    # Generate the initial blog post
    blog_post = llama(prompt_or_messages=prompt, max_tokens=4096, temperature=0.6)

    # Second LLM pass for content refinement to ensure HTML format and readability
    refinement_prompt = (
        f'''Refine the following blog post for better readability, coherence, and completeness. Remove redundancy, ensure all sentences are complete, and make it concise without losing important details. 
          Do not mention any of the changes made.The output format will be like this type <h1>title<h1/>.  Blog Post: \n {blog_post}''')

    refinement_messages = [{"role": "user", "content": refinement_prompt}]
    refined_blog_post = llama(prompt_or_messages=refinement_messages, max_tokens=4096, temperature=0.6)

    return refined_blog_post

# Function to generate trending tags based on the content
def generate_title(topic_content):
    # Define the prompt to generate tags in a plain list format
    title_prompt = (
    "Create an engaging, SEO-friendly title that captures the core message of the following content. "
    "Make it attention-grabbing and informative to appeal to online readers and maximize search visibility. "
    "Keep the title concise and relevant to the content's primary focus. without any additional words or formatting.\n\n"
    "Don't write 'Here are a few options for an engaging and SEO-friendly title ...' Don't write these type, Write only the title "
    f"Content: '{topic_content[:500]}...'"
    )


    # Format messages for generating tags
    messages = [
        {"role": "user", "content": title_prompt}
        ]

    # Generate the tags
    title_response = llama(prompt_or_messages=messages, max_tokens=20, temperature=0.9)
    
    # Process the response to ensure it's a list of tags
    #tags = [tag.strip() for tag in tags_response.split(",") if tag.strip()]

    return title_response

# Function to generate trending tags based on the content
def generate_trending_tags(topic_content):
    # Define the prompt to generate tags in a plain list format
    tags_prompt = (
        "Based on the following content, generate 5 trending , common and relevant tags that capture key aspects of the topic. "
        "The tags should be suitable for SEO and popular among online audiences interested in this topic. "
        "Return only the tags as a comma-separated list, without any additional words or formatting.\n\n"
       
        f"Content: '{topic_content[:500]}...'"
    )

    # Format messages for generating tags
    messages = [
        {"role": "user", "content": tags_prompt}
    ]

    # Generate the tags
    tags_response = llama(prompt_or_messages=messages, max_tokens=20, temperature=0.9)
    
    # Process the response to ensure it's a list of tags
    tags = [tag.strip() for tag in tags_response.split(",") if tag.strip()]

    return tags


def generate_image_prompt(topic_content):
    # Define the prompt to generate an image description
    image_prompt_instructions = (
        "Using the following content, create a realistic and visually descriptive prompt for generating an image suitable for an article or blog post. "
        "The prompt should describe the scene with natural, true-to-life details, focusing on accurate colors, textures, lighting, and setting. "
        "Include elements that convey a professional, polished look as seen in high-quality editorial photography. "
        "Avoid any abstract or overly stylized details, aiming instead for a straightforward, genuine representation. "
        "Return the prompt as a single, complete sentence without additional formatting.\n\n"
        f"Content: '{topic_content[:500]}...'"
    )
    
    # Format messages for generating the realistic image prompt
    messages = [
        {"role": "user", "content": image_prompt_instructions}
    ]
    
    # Generate the real-type image prompt
    image_prompt_response = llama(prompt_or_messages=messages, max_tokens=60, temperature=0.8)
    
    # Process the response to ensure it’s a complete sentence
    image_prompt = image_prompt_response.strip()
    
    return image_prompt

    # Format messages for generating the image prompt
    messages = [
        {"role": "user", "content": image_prompt_instructions}
    ]
    
    # Generate the image prompt
    image_prompt_response = llama(prompt_or_messages=messages, max_tokens=60, temperature=0.8)
    
    # Process the response to ensure it’s a complete sentence
    image_prompt = image_prompt_response.strip()
    
    return f"Generate a realistic image with this: {image_prompt}"



In [ ]:
import requests
import time

# Define your bot token and chat ID
TOKEN = "your_telegram_bot_token_here"  # Replace with your actual bot token
CHAT_ID = "your_chat_id_here"  # Replace with your actual chat ID

# Function to send a message with inline buttons
def send_telegram_options(top_topics):
    url = f"https://api.telegram.org/bot{TOKEN}/sendMessage"
    
    # Define options as inline buttons
    options = [
        [{"text": "Topic 1", "callback_data": "Topic 1"}],
        [{"text": "Topic 2", "callback_data": "Topic 2"}],
        [{"text": "Topic 3", "callback_data": "Topic 3"}],
        [{"text": "Topic 4", "callback_data": "Topic 4"}],
        [{"text": "Topic 5", "callback_data": "Topic 5"}],
    ]
    
    # Message with inline buttons
    data = {
        "chat_id": CHAT_ID,
        "text": f'''Please select one of the following Topic: \n\n Topic 1: \n {top_topics[0][0]}\n\n Topic 2: \n{top_topics[1][0]} \n\n Topic 3: \n {top_topics[2][0]} \n\n Topic 4: \n {top_topics[3][0]}\n\n Topic 5:\n{top_topics[4][0]} \n\n''',
        "reply_markup": {"inline_keyboard": options}
    }
    
    # Send the message with buttons
    response = requests.post(url, json=data)
    if response.status_code == 200:
        print("Options sent successfully.")
    else:
        print("Failed to send options:", response.text)

# Function to send a plain message (used to confirm selection)
def send_telegram_message(chat_id, message):
    url = f"https://api.telegram.org/bot{TOKEN}/sendMessage"
    data = {
        "chat_id": chat_id,
        "text": message
    }
    response = requests.post(url, json=data)
    return response.status_code == 200

# Clear old updates to avoid confusion
def clear_old_updates():
    url = f"https://api.telegram.org/bot{TOKEN}/getUpdates"
    response = requests.get(url)
    data = response.json()
    
    # Check if the response contains updates
    if data["ok"] and "result" in data and len(data["result"]) > 0:
        # Mark all past updates as processed
        last_update_id = data["result"][-1]["update_id"]
        requests.get(f"{url}?offset={last_update_id + 1}")
    else:
        print("No old updates to clear.")

# Function to get the user's selected option
def get_user_selected_option():
    url = f"https://api.telegram.org/bot{TOKEN}/getUpdates"
    last_update_id = None
    timeout = time.time() + 600  # 10 minutes from now

    while time.time() < timeout:
        response = requests.get(url)
        data = response.json()
        
        if data["ok"] and "result" in data:
            for update in data["result"]:
                # Skip processed updates
                if update["update_id"] == last_update_id:
                    continue
                last_update_id = update["update_id"]
                
                # Check if there's a callback query (user clicked a button)
                if "callback_query" in update:
                    callback_data = update["callback_query"]["data"]
                    # Send confirmation back to the user
                    send_telegram_message(CHAT_ID, f"You selected: {callback_data}")
                    return callback_data
        time.sleep(2)  # Check every 2 seconds

    send_telegram_message(CHAT_ID, "No selection was made within 10 minutes.")
    send_telegram_message(CHAT_ID, "Proceed with Topic 1")
    return None

# Placeholder for top topics
top_topics = [
    ["Details for Topic 1"],
    ["Details for Topic 2"],
    ["Details for Topic 3"],
    ["Details for Topic 4"],
    ["Details for Topic 5"],
]

# Clear old updates to ensure we're only processing new interactions
clear_old_updates()

# Send the options to the user
send_telegram_options(top_topics)

# Get the user's selected option
selected_option = get_user_selected_option()

# Process based on the selected option
if selected_option == "Topic 1":
    print("Processing Topic 1")
    top_topic = top_topics[0][0]
elif selected_option == "Topic 2":
    print("Processing Topic 2")
    top_topic = top_topics[1][0]
elif selected_option == "Topic 3":
    print("Processing Topic 3")
    top_topic = top_topics[2][0]
elif selected_option == "Topic 4":
    print("Processing Topic 4")
    top_topic = top_topics[3][0]
elif selected_option == "Topic 5":
    print("Processing Topic 5")
    top_topic = top_topics[4][0]
else:
    print("No valid option selected within the time limit.")
    send_telegram_message(CHAT_ID, "No valid option selected within the time limit.")
    print("Processing Topic 1")
    top_topic = top_topics[0][0]


In [ ]:

# Run the function for the top topic
  # First item in top topics list
final_blog_post = generate_blog_with_references(top_topic, data_with_topics)
title=generate_title(final_blog_post)
tag = generate_trending_tags(final_blog_post)
image_prompt=generate_image_prompt(final_blog_post)
clear_output()


In [ ]:
print(f"Title: {title}")
from IPython.display import display, Markdown
display(Markdown(final_blog_post))

In [ ]:

# Set your desired dimensions here
width = 768  # or any width you want
height = 512  # or any height you want

# Generate the image with the specified dimensions

image = pipe(prompt=image_prompt, height=height, width=width).images[0]

# Save the generated image
image.save("/kaggle/working/image.png")

telegram_message("Blog generation completed with the selected topic! 📝✨")

In [ ]:
# # # # In another cell, you can now print the blog post
# # # print("Refined Blog Post:\n", final_blog_post)
# # from IPython.display import display, Markdown
# # # Display as markdown
# display(Markdown(final_blog_post))
# # final_blog_post

In [ ]:
import markdown
html_output = markdown.markdown(final_blog_post)
print(html_output)

In [ ]:
import requests

# Function to upload an image to Imgur
def upload_image_to_imgur(image_path, client_id):
    """
    Uploads an image to Imgur and returns the image link.
    
    Parameters:
    - image_path (str): The path to the image file.
    - client_id (str): The Imgur Client ID.
    
    Returns:
    - str: The URL of the uploaded image.
    """
    headers = {"Authorization": f"Client-ID {client_id}"}
    with open(image_path, "rb") as image_file:
        response = requests.post(
            "https://api.imgur.com/3/upload",
            headers=headers,
            files={"image": image_file}
        )
    # Check if the response is successful
    if response.status_code == 200:
        # Get the link from the response
        link = response.json().get('data', {}).get('link', 'No link found')
        return link
    else:
        print("Failed to upload image:", response.text)
        return None

# Example usage
client_id = "your_imgur_client_id_here"  # Replace with your actual Imgur Client ID
image_path = "path_to_your_image.png"  # Replace with the path to your image
image_url = upload_image_to_imgur(image_path, client_id)

if image_url:
    print(f"Image URL: {image_url}")
else:
    print("Image upload failed.")


In [ ]:
# # Format
# import re
# title=str(title)
# # Remove symbols only from the start and end
# cleaned_title = re.sub(r'^[^\w]+|[^\w]+$', '', title)


def add_banner_to_html(html_output, image_url):
    # Parse the HTML
    soup = BeautifulSoup(html_output, 'html.parser')
    
    # Check for the first <h1> tag
    h1_tag = soup.find("h1")
    
    if h1_tag:
        # Create the <figure> with an <img> tag after the <h1> tag
        figure_tag = soup.new_tag("figure")
        img_tag = soup.new_tag("img", src=image_url)
        figure_tag.append(img_tag)
        h1_tag.insert_after(figure_tag)
    else:
        # If no <h1> tag, find the first <strong> tag
        strong_tag = soup.find("strong")
        if strong_tag:
            # Convert <strong> to <h1>
            h1_tag = soup.new_tag("h1")
            h1_tag.string = strong_tag.string  # Transfer the text content
            strong_tag.replace_with(h1_tag)
            
            # Create the <figure> with an <img> tag after the new <h1> tag
            figure_tag = soup.new_tag("figure")
            img_tag = soup.new_tag("img", src=image_url)
            figure_tag.append(img_tag)
            h1_tag.insert_after(figure_tag)
    
    # Convert the soup back to a string with the modifications
    modified_html = str(soup)
    return modified_html
body_html=add_banner_to_html(html_output,image_url)
print(body_html)

In [ ]:
import requests
import json

# Replace with your Medium Integration Token
MEDIUM_INTEGRATION_TOKEN = 'your_medium_integration_token_here'

# API URLs
MEDIUM_API_BASE_URL = 'https://api.medium.com/v1'
USER_INFO_URL = f"{MEDIUM_API_BASE_URL}/me"
CREATE_POST_URL = f"{MEDIUM_API_BASE_URL}/users/{{user_id}}/posts"

def get_user_id():
    """
    Fetches the user ID associated with the Medium Integration Token.
    
    Returns:
    - str: The user ID if successful, None otherwise.
    """
    headers = {
        "Authorization": f"Bearer {MEDIUM_INTEGRATION_TOKEN}"
    }
    response = requests.get(USER_INFO_URL, headers=headers)
    
    if response.status_code == 200:
        user_info = response.json()
        return user_info['data']['id']
    else:
        print("Error fetching user information:", response.text)
        return None

def create_medium_post(user_id, title, content, content_format='html', tags=None, publish_status='public'):
    """
    Creates a new post on Medium.
    
    Parameters:
    - user_id (str): The ID of the user creating the post.
    - title (str): The title of the post.
    - content (str): The content of the post in the specified format.
    - content_format (str): Format of the content, 'html' or 'markdown'.
    - tags (list): A list of tags for the post.
    - publish_status (str): The publishing status ('public', 'draft', or 'unlisted').
    
    Returns:
    - dict: The response JSON if successful, None otherwise.
    """
    headers = {
        "Authorization": f"Bearer {MEDIUM_INTEGRATION_TOKEN}",
        "Content-Type": "application/json",
        "Accept": "application/json",
    }

    data = {
        "title": title,
        "contentFormat": content_format,
        "content": content,
        "tags": tags or [],
        "publishStatus": publish_status
    }

    response = requests.post(CREATE_POST_URL.format(user_id=user_id), headers=headers, data=json.dumps(data))
    
    if response.status_code == 201:
        print("Post created successfully!")
        return response.json()
    else:
        print("Error creating post:", response.text)
        return None

# Example usage
user_id = get_user_id()
if user_id:
    # Create Medium post
    response = create_medium_post(user_id, title=title, content=body_html, content_format="html", tags=tags)

    if response:
        telegram_message("Blog successfully posted on Medium! 🚀📘")


In [ ]:
import boto3
import requests
import time
from google.oauth2.credentials import Credentials
from google.auth.transport.requests import Request
from googleapiclient.discovery import build

# AWS S3 configurations
BUCKET_NAME = "your_bucket_name_here"  # Replace with your S3 bucket name
TOKEN_FILE_PATH = "refresh_token.txt"  # Path in S3 to store the refresh token

# OAuth and Blogger configurations
CLIENT_ID = "your_google_client_id_here"  # Replace with your Google Client ID
CLIENT_SECRET = "your_google_client_secret_here"  # Replace with your Google Client Secret
BLOG_ID = "your_blog_id_here"  # Replace with your Blogger blog ID
SCOPES = ["https://www.googleapis.com/auth/blogger"]

# Telegram bot credentials
TOKEN = "your_telegram_bot_token_here"  # Replace with your Telegram bot token
CHAT_ID = "your_chat_id_here"  # Replace with your Telegram chat ID

# Initialize the AWS S3 client
session = boto3.Session(
    aws_access_key_id="your_aws_access_key_id_here",  # Replace with your AWS Access Key ID
    aws_secret_access_key="your_aws_secret_access_key_here",  # Replace with your AWS Secret Access Key
    region_name="your_aws_region_here"  # Replace with your AWS region
)
s3_client = session.client("s3")

def save_token_to_s3(token):
    """Saves the refresh token to S3."""
    s3_client.put_object(Bucket=BUCKET_NAME, Key=TOKEN_FILE_PATH, Body=token)
    print("Refresh token saved to S3.")

def load_token_from_s3():
    """Loads the refresh token from S3."""
    try:
        response = s3_client.get_object(Bucket=BUCKET_NAME, Key=TOKEN_FILE_PATH)
        return response["Body"].read().decode("utf-8")
    except s3_client.exceptions.NoSuchKey:
        print("Token file not found in S3.")
        send_telegram_msg(CHAT_ID, "Token file not found in S3.")
        return None

# Function to send a message through Telegram
def send_telegram_msg(chat_id, message):
    url = f"https://api.telegram.org/bot{TOKEN}/sendMessage"
    data = {"chat_id": chat_id, "text": message}
    requests.post(url, json=data)

# Function to get the latest refresh token from the user (if needed)
def get_new_refresh_token():
    url = f"https://api.telegram.org/bot{TOKEN}/getUpdates"
    last_update_id = None
    start_time = time.time()

    while time.time() - start_time < 600:  # Wait up to 10 minutes
        response = requests.get(url)
        data = response.json()

        if data["ok"] and "result" in data:
            for update in data["result"]:
                if update["update_id"] == last_update_id:
                    continue
                last_update_id = update["update_id"]

                if "message" in update and update["message"]["chat"]["id"] == int(CHAT_ID):
                    text = update["message"]["text"]
                    return text
        time.sleep(2)

    return None

# Function to get credentials
def get_credentials():
    global REFRESH_TOKEN
    try:
        creds = Credentials(
            None,
            refresh_token=REFRESH_TOKEN,
            client_id=CLIENT_ID,
            client_secret=CLIENT_SECRET,
            token_uri="https://oauth2.googleapis.com/token",
            scopes=SCOPES
        )
        
        if not creds.valid:
            creds.refresh(Request())
        return creds

    except Exception as e:
        print("Token refresh failed:", e)
        send_telegram_msg(CHAT_ID, "Your refresh token is invalid. Please provide a new one within 10 minutes.")
        
        # Request a new refresh token via Telegram
        new_refresh_token = get_new_refresh_token()
        if new_refresh_token:
            REFRESH_TOKEN = new_refresh_token
            save_token_to_s3(new_refresh_token)
            return get_credentials()
        else:
            print("No new token provided within the time limit.")
            send_telegram_msg(CHAT_ID, "No new token provided within the time limit.")
            return None

# Function to create a blog post
def create_blog_post(credentials, title, content):
    service = build("blogger", "v3", credentials=credentials)
    
    post = {
        "title": title,
        "content": content
    }

    post_result = service.posts().insert(blogId=BLOG_ID, body=post).execute()
    print(f"Post published: {post_result['url']}")
    send_telegram_msg(CHAT_ID, "Blog successfully posted on Blogger! 📝✅")

# Main function to execute
def main():
    credentials = get_credentials()
    if credentials:
        create_blog_post(credentials, title, content=body_html)
    else:
        print("Credentials could not be obtained. Exiting.")
        send_telegram_msg(CHAT_ID, "Credentials could not be obtained. Exiting.")

# Run the main function
if __name__ == "__main__":
    main()


In [ ]:
telegram_message("All processes completed! 🎉 Thank you for using Bloggy! 😊")